In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import numpy as np
from sklearn.metrics import classification_report

from sklearn.metrics import accuracy_score


import warnings
from pyod.models.auto_encoder import AutoEncoder
from pyod.utils.data import evaluate_print
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from scipy.stats import ConstantInputWarning

warnings.simplefilter("ignore", ConstantInputWarning)


In [2]:
df=pd.read_csv("/Users/saikarthik/Desktop/Nexus-S1 Deakin /datasets /cleaned_LEAD.csv")

In [3]:
df.drop(columns=['Unnamed: 0'], inplace=True)


In [5]:
scaler = StandardScaler()
X = df[['meter_reading']].values
X_scaled = scaler.fit_transform(X)
X=X_scaled

In [6]:
clf_ae = AutoEncoder(hidden_neuron_list=[8, 4, 4, 8], epoch_num=50, batch_size=32, contamination=0.1)


In [7]:
clf_ae.fit(X_scaled)

Training:   0%|          | 0/50 [00:00<?, ?it/s]

: 

In [ ]:
y_predict_ae = clf_ae.predict(X_scaled)
y_scores_ae = clf_ae.decision_scores_

In [ ]:
print(classification_report(df['anomaly'], y_predict_ae))
accuracy = accuracy_score(df['anomaly'], y_predict_ae)

In [ ]:
accuracy*100